In [10]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


In [12]:
Train = pd.read_csv("Train_Set.csv")
Test = pd.read_csv("Test_Set.csv")

Train.drop(['Unnamed: 0'], axis=1, inplace=True)
Test.drop(['Unnamed: 0'], axis=1, inplace=True)

MemoryError: Unable to allocate 8.00 KiB for an array with shape (1024,) and data type int64

In [ ]:
label_cols = ["label", "binned_label"]

X_Train = Train.drop(columns=label_cols)
y_binned_Train = Train['binned_label']
y_Train = Train['label']

X_Test = Test.drop(columns=label_cols)
y_binned_Test = Test['binned_label']
y_Test = Test['label']

In [ ]:
int64_cols = Train.shape[1] - 6
cols_convert = Train.columns[:int64_cols]
dtype_dict = {col: np.int8 for col in cols_convert}
Train[cols_convert] = Train[cols_convert].astype(dtype_dict)

int64_cols = Test.shape[1] - 6
cols_convert = Test.columns[:int64_cols]
dtype_dict = {col: np.int8 for col in cols_convert}
Test[cols_convert] = Test[cols_convert].astype(dtype_dict)

In [ ]:
KF = KFold(n_splits=5)
def Model_Train(Model, X, Y):
    Preds = []
    for i,j in KF.split(X):
        Train_X = X.iloc[i]
        Test_X = X.iloc[j]
        Train_Y = Y.iloc[i]
        
        Model.fit(Train_X, Train_Y)
        Pred_Y = Model.predict(Test_X)
        Preds.append(Pred_Y)
    print(classification_report(Y, Pred_Y))

In [ ]:
Cart = DecisionTreeClassifier()
Model_Train(Cart, X_Train, y_binned_Train)

y_pred = Cart.predict(X_Test)
print("Metrics: \n", classification_report(y_binned_Test, y_pred))

Metrics: 
                  precision    recall  f1-score   support

1425k - 875000k       0.99      0.99      0.99     23365
    188k - 373k       0.98      0.99      0.99     58034
       1k - 89k       1.00      1.00      1.00     23646
    374k - 699k       0.99      0.99      0.99     58602
   700k - 1424k       0.99      0.99      0.99     35088
     90k - 187k       0.99      0.98      0.99     35385

       accuracy                           0.99    234120
      macro avg       0.99      0.99      0.99    234120
   weighted avg       0.99      0.99      0.99    234120



In [ ]:
with open("Cart.pkl", "wb") as f:
    pickle.dump(Cart, f)